In [ ]:
###########import packages##########
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.backend.tensorflow_backend import set_session
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import GridSearchCV
%matplotlib

#GPU设置
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
###########fix random seed for reproducability##########
seed=1
np.random.seed(seed)
###########wrapping root mean square error for later calls##########
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
###########loading data##########
csv = "../../database/database124.csv"

fdata=pd.read_csv(csv, index_col=0)
raw_data = fdata.iloc[:, 1:]

median_raw_data=raw_data.median()
dict_median_raw_data=median_raw_data.to_dict()
data=raw_data.fillna(dict_median_raw_data)
#print(data)
#print(np.std(data,axis=0))
###########data standardization##########
standardized_data = (data-np.mean(data,axis=0))/np.std(data,axis=0)
raw_param=standardized_data.iloc[:,0:121]
raw_power=standardized_data.iloc[:,121]
#print(raw_param)
print(raw_power)
X=raw_param.values.astype(np.float32)
y=raw_power.values.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15)
###########defining a wrapper function for later call from each machine learning algorithms##########
def try_different_method(model):
    model.fit(X_train,y_train)
    score = model.score(X_test, y_test)
    result = model.predict(X_test)
    x_prediction_maximum_power_ann=result*np.std(data,axis=0)[121]+np.mean(data,axis=0)[121]
    y_real_maximum_power=y_test*np.std(data,axis=0)[121]+np.mean(data,axis=0)[121]
    print(x_prediction_maximum_power_ann)
    x_prediction_maximum_power_ann_series=pd.Series(x_prediction_maximum_power_ann)
    y_real_maximum_power_series=pd.Series(y_real_maximum_power)
    ###########evaluating the regression quality##########
    corr_ann = round(x_prediction_maximum_power_ann_series.corr(y_real_maximum_power_series), 4)
    rmse_val= rmse(x_prediction_maximum_power_ann,y_real_maximum_power)
    print(rmse_val)
    print(corr_ann)
    print(y_real_maximum_power)
    ###########generating a figure##########
    x_y_x=np.arange(0,10000,100)
    x_y_y=np.arange(0,10000,100)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.scatter(x_prediction_maximum_power_ann,y_real_maximum_power,color='red',label='Artificial Neural Network')
    plt.legend()
    ax.plot(x_y_x,x_y_y)
    plt.xlabel(u"Predicted_Time (h)")
    plt.ylabel(u"Real_Time (h)")
    plt.show()
####import machine learning algorithms packages and define the corresponding models####
####Support Vector Regressor####
from sklearn import svm
model_SVR = svm.SVR()

####Random Forest####
from sklearn import ensemble
model_RandomForestRegressor = ensemble.RandomForestRegressor()

####Gaussian Process####
from sklearn.gaussian_process import GaussianProcessRegressor
model_GaussianProcessRegressor=GaussianProcessRegressor()

####XGBoost####
import xgboost as xgb
model_XGboostRegressor=xgb.XGBRegressor()

In [ ]:
seed=2
np.random.seed(seed)
####Gaussian Process Regressor####

algorithm_name='Gaussian Process Regressor'
try_different_method(model_GaussianProcessRegressor)

In [ ]:
seed=2
np.random.seed(seed)
####Support Vector Regressor####
###########defining the parameters dictionary##########
param = {'kernel':['rbf','linear','poly'],
         'max_iter':[100,200,300,400,500,600,700,800,1000,1100,1200,1300,1400,1500],
         'degree':[2,3,4],
         'gamma':['auto'],
         'epsilon':[0.001,0.01,0.1,0.3,0.5,0.7,1],
         'coef0':[100,200,300,400,500,600,700,800,1000,1100,1200,1300,1400,1500]
       }
grid = GridSearchCV(model_SVR,param_grid=param,cv=5)
grid.fit(X_train,y_train)
print('Best Regressor:',grid.best_params_,'Best Score:', grid.best_score_) 
best_model=grid.best_estimator_

algorithm_name='Support Vector Regressor'
try_different_method(best_model)

In [ ]:
seed=2
np.random.seed(seed)
####Random Forest####
###########defining the parameters dictionary##########
param = {'n_estimators':[200,400,800],
         'max_depth':[5,10,15],
         'min_samples_split':[2,4,6,8],
         'min_samples_leaf':[2,4,8],
         'max_features':['auto','sqrt','log2']

       }

grid = GridSearchCV(model_RandomForestRegressor,param_grid=param,cv=5)
grid.fit(X_train,y_train)
print('Best Regressor:',grid.best_params_,'Best Score:', grid.best_score_) 
best_model=grid.best_estimator_

algorithm_name='Random Forest Regressor'
try_different_method(best_model)

In [ ]:
seed=2
np.random.seed(seed)
####XGboost####
###########defining the parameters dictionary##########
param = {
         'learning_rate':[0.01,0.1,1],
   'n_estimators':[100,200,400,1000,2000,4000,8000],
    'max_depth':[5,7,9,11],  
    'objective':['reg:squarederror'],
    'subsample':[0.5,0.6,0.7,0.8,0.9],
    'lambda':[0.1],
    'alpha':[0.1]
       }
grid = GridSearchCV(model_XGboostRegressor,param_grid=param,cv=5)
grid.fit(X_train,y_train)
print('Best Regressor:',grid.best_params_,'Best Score:', grid.best_score_) 
best_model=grid.best_estimator_

algorithm_name='XGBoost Regressor'
try_different_method(best_model)